In [ ]:
# -----------------------------
# 🔧 1. Ghost MLP Block (ConvNeXt-style)
# -----------------------------
class GhostMLPBlock(nn.Module):
    def __init__(self, in_channels, expansion_ratio=4):
        super(GhostMLPBlock, self).__init__()
        hidden_dim = in_channels * expansion_ratio
        self.fc1 = GhostModule(in_channels, hidden_dim, kernel_size=1)
        self.act = nn.GELU()
        self.fc2 = GhostModule(hidden_dim, in_channels, kernel_size=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.fc2(x)
        return x

# -----------------------------
# 🔧 2. Modified ConvNeXt-Attention Block
# -----------------------------
class HybridConvNeXtBlock(nn.Module):
    def __init__(self, channels, attention_module):
        super(HybridConvNeXtBlock, self).__init__()
        self.dwconv = nn.Conv2d(channels, channels, kernel_size=7, padding=3, groups=channels)  # Positional encoding
        self.norm1 = nn.LayerNorm(channels)
        self.attn = attention_module(channels)  # Could be MHSA, Axial, etc.
        self.norm2 = nn.LayerNorm(channels)
        self.mlp = GhostMLPBlock(channels)

    def forward(self, x):
        residual = x
        x = self.dwconv(x)
        x = x.permute(0, 2, 3, 1)  # (N, C, H, W) -> (N, H, W, C)
        x = self.norm1(x)
        x = self.attn(x)
        x = x + residual.permute(0, 2, 3, 1)

        residual = x
        x = self.norm2(x)
        x = self.mlp(x.permute(0, 3, 1, 2))
        x = x.permute(0, 2, 3, 1) + residual
        return x.permute(0, 3, 1, 2)  # (N, H, W, C) -> (N, C, H, W)

# -----------------------------
# 🔧 3. Ghost CoordAttention Block
# -----------------------------
class GhostCoordAttention(nn.Module):
    def __init__(self, inp, reduction=32):
        super(GhostCoordAttention, self).__init__()
        self.pool_h = nn.AdaptiveAvgPool2d((None, 1))
        self.pool_w = nn.AdaptiveAvgPool2d((1, None))

        mid = max(8, inp // reduction)
        self.conv1 = GhostModule(inp, mid, kernel_size=1)
        self.bn1 = nn.BatchNorm2d(mid)
        self.act = nn.ReLU()
        self.conv_h = GhostModule(mid, inp, kernel_size=1)
        self.conv_w = GhostModule(mid, inp, kernel_size=1)

    def forward(self, x):
        identity = x
        n, c, h, w = x.size()

        x_h = self.pool_h(x).permute(0, 1, 3, 2)
        x_w = self.pool_w(x)
        y = torch.cat([x_h, x_w], dim=2)
        y = self.conv1(y)
        y = self.bn1(y)
        y = self.act(y)

        x_h, x_w = torch.split(y, [h, w], dim=2)
        x_h = self.conv_h(x_h.permute(0, 1, 3, 2))
        x_w = self.conv_w(x_w)

        out = identity * torch.sigmoid(x_h) * torch.sigmoid(x_w)
        return out

# -----------------------------
# 🔧 4. Ghost ASPPFELAN Block (Simplified)
# -----------------------------
class GhostASPPFELAN(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GhostASPPFELAN, self).__init__()
        self.branch1 = GhostModule(in_channels, out_channels, kernel_size=1)
        self.branch2 = GhostModule(in_channels, out_channels, kernel_size=3, dilation=2, padding=2)
        self.branch3 = GhostModule(in_channels, out_channels, kernel_size=3, dilation=4, padding=4)
        self.fuse = GhostModule(out_channels * 3, out_channels, kernel_size=1)

    def forward(self, x):
        b1 = self.branch1(x)
        b2 = self.branch2(x)
        b3 = self.branch3(x)
        x = torch.cat([b1, b2, b3], dim=1)
        x = self.fuse(x)
        return x

# -----------------------------
# 🏗️ Full Architecture (Pseudo)
# -----------------------------
class ASPPNeXt(nn.Module):
    def __init__(self):
        super(ASPPNeXt, self).__init__()
        self.encoder_stages = nn.ModuleList([
            HybridConvNeXtBlock(96, attention_module),
            HybridConvNeXtBlock(192, attention_module),
            HybridConvNeXtBlock(384, attention_module),
            HybridConvNeXtBlock(768, attention_module)
        ])

        self.bottleneck = DAAFModule(768)  # Assume DAAFModule is implemented

        self.decoder_stages = nn.ModuleList([
            GhostASPPFELAN(768, 384),
            GhostASPPFELAN(384, 192),
            GhostASPPFELAN(192, 96)
        ])

        self.attentions = nn.ModuleList([
            GhostCoordAttention(384),
            GhostCoordAttention(192),
            GhostCoordAttention(96)
        ])

        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.final_conv = nn.Conv2d(96, num_classes, kernel_size=1)

    def forward(self, x):
        enc_feats = []
        for stage in self.encoder_stages:
            x = stage(x)
            enc_feats.append(x)

        x = self.bottleneck(x)

        for i in range(3):
            x = self.upsample(x)
            x = x + enc_feats[-(i+2)]  # Skip connection
            x = self.decoder_stages[i](x)
            x = self.attentions[i](x)

        x = self.upsample(x)
        return self.final_conv(x)

# Note: GhostModule and DAAFModule need to be implemented or imported


In [ ]:
--------------------xX DAAF Block with GhostModule Integrations Xx--------------------

Inputs:
  f_rgb_preGhost ∈ ℝ^(B×C×H×W)
  f_depth_preGhost ∈ ℝ^(B×C×H×W)

1. Local Branch (RDSCB + LIA)
   ┌────────────────────────────────────┐
   │ RDSCB (n∈{1,3,5,7}):               │
   │   For each n: GhostConv(n×n)       │
   │   LeakyReLU                        │
   │ Concatenate → GhostConv(1×1)       │
   └────────────────────────────────────┘
       ↓
   LIA: Conv1D pooling → GhostConv(1×1)

2. Global Branch (ITB)
   ┌────────────────────────────────────┐
   │ Interactive Self-Attention (ISA)   │
   │   Cross-modal MSA on (f_rgb, f_d)  │
   │ Residual Add → LayerNorm           │
   │ GhostConv FFN:                     │
   │   • GhostConv(1×1): C→4C           │
   │   • GELU                           │
   │   • GhostConv(1×1): 4C→C           │
   │ Residual Add                       │
   └────────────────────────────────────┘

3. Fusion and Reconstruction
   ┌──────────────────────────────────────────┐
   │ Concat(local, global_rgb, global_depth)  │
   │ GhostConv(3×3) → LeakyReLU               │
   │ (This is the “Global Fusion” Conv)       │
   └──────────────────────────────────────────┘
       ↓
   Reconstruction Head:
   ┌────────────────────────────────────┐
   │ GhostConv(3×3) → LeakyReLU         │
   │ GhostConv(3×3) → LeakyReLU         │
   │ GhostConv(3×3) → LeakyReLU         │
   └────────────────────────────────────┘
       ↓
   Output: f_fused ∈ ℝ^(B×C×H×W)
